# Bermudan Swaption Pricing

In this notebook we show how Bermudan swaption can be set up and prices. 

A Bermudan swaption allows the option holder to enter into a swap at several exercise times. For each individual exercise time the option represents a European swaption. Thus it natural to describe a Bermudan swaption by a list of (co-terminal) European swaptions with equal strike and swap details.

If exercised the swap payoff can also be represented as a stream of deterministic cash flows. Consequently, we can equivalently view the Bermudan swaption as a Bermudan bond option. And for the Bermudan bond option representation we can then apply our backward induction algorithm with the Hull White model.

In [ ]:
import sys
sys.path.append('../') # make sure we can access the src/ folder

from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import QuantLib as ql
from tqdm import tqdm

from src.hull_white_model import HullWhiteModel
from src.swaption import create_swaption
from src.yieldcurve import YieldCurve

## Market Data

In this example we want to focus on the Bermudan swaption setup and pricing. Consequently, we set up simplified market data with flat forward rate and flat market volatilities.

In [ ]:
yield_curve       = YieldCurve(['70y'], [0.03])
market_volatility = 100 * 1e-4  # 100bp
mean_reversion    = 0.05

## Bermudan Swaption Specification

We consider a Bermudan swaption with 20y maturity and annual exercise times.

In [ ]:
maturity = 20 # in years
expiry_terms = [ str(e)+'y' for e in range(1, maturity) ]
swap_terms   = [ str(maturity - e)+'y' for e in range(1, maturity) ]
for e, s in zip(expiry_terms, swap_terms):
    print(e + '-' + s)

For each exercise time we create a European swaption instrument with a given strike (of 3%).

In [ ]:
swaptions = [
    create_swaption(e, s, yield_curve, yield_curve, strike=0.03, normalVolatility=market_volatility)
    for e, s in zip(expiry_terms, swap_terms)
]

## Model Calibration

Our Hull White model should ensure that model-implied prices of all European swaptions are consistent to market prices (or Vanilla model prices) of that European swaptions.

Consequently, we calibrate our Hull White model for Bermudan pricing to the corresponding European swaptions.

For this step we re-use the product-specific bootstrapping method from *HullWhiteModelCalibation.ipynb*.

In [ ]:
from scipy.optimize import brentq

def model_from_swaptions(european_swaptions, yield_curve, mean_reversion):
    details  = [ s.bond_option_details() for s in european_swaptions ]
    ref_npv  = [ s.npv_via_bachelier()   for s in european_swaptions ]
    ref_vega = [ s.vega()                for s in european_swaptions ]
    #
    volatility_times  = np.array([ d['expiry_time'] for d in details ])
    volatility_values = np.zeros(volatility_times.shape)
    for idx in tqdm(range(len(european_swaptions)), 'Model calibration'):
        def obj(sigma):
            volatility_values[idx:] = sigma
            model = HullWhiteModel(yield_curve, mean_reversion, volatility_times, volatility_values)
            model_npv = model.coupon_bond_option(
                details[idx]['expiry_time'],
                details[idx]['pay_times'],
                details[idx]['cash_flows'],
                details[idx]['strike_price'],
                details[idx]['call_or_put']
                )
            return (model_npv - ref_npv[idx]) / ref_vega[idx]
        vol_guess = european_swaptions[idx].normalVolatility
        sigma_idx = brentq(obj, 0.1*vol_guess, 5.0*vol_guess, xtol=1.0e-6)
        volatility_values[idx] = sigma_idx
    return HullWhiteModel(yield_curve, mean_reversion, volatility_times, volatility_values)

In [ ]:
model = model_from_swaptions(swaptions, yield_curve, mean_reversion=mean_reversion)

## Bermudan Option Representation

The Bermudan swaption needs to be represented as Bermudan bond option. For this step we need to calculate the exercise times and underlying payoffs.

In order for the payoff to be evaluated it needs to know the Hull White model. Therefore we use the calibrated model for Bermudan swaption setup.


In [ ]:
from src.methods.payoffs import CouponBond

def bond_option_details(european_swaptions, model):
    details  = [ s.bond_option_details() for s in european_swaptions ]
    exercise_times = np.array([ d['expiry_time'] for d in details ])
    underlying_payoffs = [
        CouponBond(model, d['expiry_time'], d['pay_times'], d['cash_flows'] * d['call_or_put'])
        for d in details
    ]
    return {
        'exercise_times' : exercise_times,
        'underlying_payoffs' : underlying_payoffs,
    }

In [ ]:
option = bond_option_details(swaptions, model)
option

## Bermudan Pricing

For Bermudan option pricing it remains to specify a pricing method. The various pricing methods are analysed in *BermudanBondOption.ipynb*. Here, we use the PDE solver because it is rather efficient if the option exhibits many exercise times.

In [ ]:
from src.methods.pde_solver import PdeSolver

method = PdeSolver(model)

Finally, we can use backward induction algorithm to price the Bermudan option.

In [ ]:
from src.bermudan_option import bermudan_option_npv

npv = bermudan_option_npv(option['exercise_times'], option['underlying_payoffs'], method, showProgress=True)
npv

We compare the resulting Bermudan option price to the co-terminal European option prices.

In [ ]:
labels = [ e+'-'+s for e, s in zip(expiry_terms, swap_terms) ] + ['Berm']
npvs   = [ s.npv() for s in swaptions                        ] + [ npv  ]

x = 4 * np.linspace(1,20,20)
fig = plt.figure(figsize=(8, 4))
plt.bar(x, npvs, 3.0)
plt.xticks(x, labels, rotation='vertical')
plt.xlabel('swaption')
plt.ylabel('option price')
plt.show()

## Bermudan Pricing Analysis

In order to conveniently analyse and compare Bermudan option pricing we wrap option setup, model calibration, pricing and plotting into a function.

In [ ]:
def bermudan_pricing_analysis(
    maturity,
    strike, 
    rate, 
    market_volatility, 
    mean_reversion, 
    method_from_model,
    show_plots = True
    ):
    #
    yield_curve = YieldCurve(['70y'], [rate])
    expiry_terms = [ str(e)+'y' for e in range(1, maturity) ]
    swap_terms   = [ str(maturity - e)+'y' for e in range(1, maturity) ]
    swaptions = [
        create_swaption(e, s, yield_curve, yield_curve, strike=strike, normalVolatility=market_volatility, payerOrReceiver=ql.VanillaSwap.Receiver)
        for e, s in zip(expiry_terms, swap_terms)
    ]
    model = model_from_swaptions(swaptions, yield_curve, mean_reversion)
    option = bond_option_details(swaptions, model)
    method = method_from_model(model)
    berm_npv = bermudan_option_npv(option['exercise_times'], option['underlying_payoffs'], method, showProgress=True)
    #
    european_npvs = [ s.npv() for s in swaptions ]
    switch_value = berm_npv - np.max(european_npvs)
    #
    if not show_plots:
        return switch_value
    #
    times = np.linspace(0.0, maturity, 100*maturity + 1)
    vols  = np.array([ model.sigma(t) for t in times ])
    fig = plt.figure(figsize=(8, 5))
    plt.plot(times, vols * 1e+4)
    plt.xlabel(r'time $t$')
    plt.ylabel(r'short rate volatility $\sigma(t)$ (bp)')
    plt.title(r'mean reversion $a=%.2f$' % mean_reversion)
    plt.ylim((0, 160))
    #
    labels = [ e+'-'+s for e, s in zip(expiry_terms, swap_terms) ] + ['Berm'    ]
    npvs   = european_npvs                                         + [ berm_npv ]    
    x = 4 * np.linspace(1, maturity, maturity)
    fig = plt.figure(figsize=(8, 4))
    plt.bar(x, npvs, 3.0)
    plt.xticks(x, labels, rotation='vertical')
    plt.xlabel('swaption')
    plt.ylabel('option price')
    plt.title(r'switch option value %.2f' % switch_value)
    plt.show()

Now, we can compare calibrated volatilities and option prices depending on mean reversion and option moneyness.

### Out-of-the-money Options

In [ ]:
bermudan_pricing_analysis(maturity=20, strike=0.03, rate=0.05, market_volatility=0.01, mean_reversion=-0.05, method_from_model=PdeSolver)


In [ ]:
bermudan_pricing_analysis(maturity=20, strike=0.03, rate=0.05, market_volatility=0.01, mean_reversion=0.05, method_from_model=PdeSolver)

### In-the-money Options

In [ ]:
bermudan_pricing_analysis(maturity=20, strike=0.03, rate=0.01, market_volatility=0.01, mean_reversion=-0.05, method_from_model=PdeSolver)

In [ ]:
bermudan_pricing_analysis(maturity=20, strike=0.03, rate=0.01, market_volatility=0.01, mean_reversion=0.05, method_from_model=PdeSolver)

## Switch Option Value Analysis

The switch option value is the difference between Bermudan model price and the maximum European option price. The switch option value is always non-negative because, by construction, the Bermudan option value must be larger or equal to the maximum Europen option price

If the Hull White model is calibrated to co-terminal European swaptions then then we can use mean reversion to control the switch option value.

We calculate the switch option value for in-the-money (ITM), at-the-money (ATM) and out-of-the-money (OTM) options.

In [ ]:
strike = 0.03

options = [
    { 'label' : 'ITM', 'rate' : 0.01 },
    { 'label' : 'ATM', 'rate' : 0.03 },
    { 'label' : 'OTM', 'rate' : 0.05 },        
]

mean_reversions = np.linspace(-0.05, 0.11, 9)

switch_value = lambda rate, mean_reversion : \
    bermudan_pricing_analysis(maturity=20, strike=0.03, rate=rate, market_volatility=0.01, mean_reversion=mean_reversion, method_from_model=PdeSolver, show_plots=False)

switch_values = []
for mean_reversion in mean_reversions:
    row = { 'mean_reversion' : mean_reversion }
    for option in options:
        row[option['label']] = switch_value(option['rate'], mean_reversion)
    switch_values.append(row)
switch_values = pd.DataFrame(switch_values)
switch_values

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(switch_values['mean_reversion'], switch_values['ITM'], label='ITM')
plt.plot(switch_values['mean_reversion'], switch_values['ATM'], label='ATM')
plt.plot(switch_values['mean_reversion'], switch_values['OTM'], label='OTM')
plt.legend()
plt.xlabel('mean reversion $a$')
plt.ylabel('switch option value')